In [ ]:
# DEMO

In [ ]:
import io
import requests
import json
import pandas as pd
from ipywidgets import HTML, Layout, IntSlider, jslink
from ipyleaflet import Map, Marker, Popup, LayerGroup, Icon, ScaleControl, LayersControl, Layer, FullScreenControl, GeomanDrawControl, WidgetControl
import plotly.express as px

import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=DeprecationWarning)
pd.set_option('mode.chained_assignment', None)


def get_from_url(url):
    s = requests.get(url).content
    return io.StringIO(s.decode('utf-8')).getvalue()

def get_basemap(centerLat, centerLon, zoom_level=6):
    m = Map(center=(centerLat, centerLon), zoom=zoom_level,  layout=Layout(width='800px', height='800px', keyboard=False))
    m.add(FullScreenControl())

    draw_control = GeomanDrawControl()
    draw_control.polyline =  {
        "pathOptions": {
            "color": "#6bc2e5",
            "weight": 8,
            "opacity": 1.0
        }
    }
    draw_control.polygon = {
        "pathOptions": {
            "fillColor": "#6be5c3",
            "color": "#6be5c3",
            "fillOpacity": 1.0
        }
    }
    draw_control.rectangle = {
        "pathOptions": {
            "fillColor": "#fca45d",
            "color": "#fca45d",
            "fillOpacity": 1.0
        }
    }
    
    #m.add(draw_control)

    zoom_slider = IntSlider(description='Zoom level:', min=0, max=15, value=zoom_level)
    jslink((zoom_slider, 'value'), (m, 'zoom'))
    widget_control1 = WidgetControl(widget=zoom_slider, position='bottomleft')
    m.add(widget_control1)
    return m


def show_tsunami_data_on_a_map(df):
    centerLat=df['properties.PlaceLat'].mean()
    centerLon=df['properties.PlaceLon'].mean()
    m = get_basemap(centerLat, centerLon)
    icon1 = Icon(icon_url='https://www.ics-c.epos-eu.org/assets/img/logo/TSU_logo.png', icon_size=[40, 40])
    markers = []
    for _, row in df.iterrows():
        popup_html = f"""
        <b>{row['properties.PlaceName']} ({row['properties.Province']})</b><br>
        Municipality: {row['properties.Municipality']} <br>
        ISTAT: {row['properties.ISTAT']}<br>
        <a href="{row['properties.Link']}" target="_blank">Link</a>
        """
        marker = Marker(location=(row['properties.PlaceLat'], row['properties.PlaceLon']), draggable=False)
        marker.icon=icon1
        marker.popup=HTML(popup_html)
        markers.append(marker)

    layer_group = LayerGroup(layers=markers)
    m.add_layer(layer_group)
    return m

def show_volcano_data_on_a_map(df, zoom_level=11):
    centerLat=df['lat'].mean()
    centerLon=df['lon'].mean()
    m = get_basemap(centerLat, centerLon, zoom_level=zoom_level)
    icon1 = Icon(icon_url='https://www.ics-c.epos-eu.org/assets/img/logo/VOLCANO_logo.png', icon_size=[40, 40])
    markers = []
    for _, row in df.iterrows():
        popup_content = "".join(
            f"<b>{col}:</b> {row[col]}<br>"
            for col in df.columns if col not in ['Lat', 'Lon']
        )
        marker = Marker(location=(row['lat'], row['lon']), draggable=False)
        marker.icon=icon1
        marker.popup=HTML(popup_content)
        markers.append(marker)

    layer_group = LayerGroup(layers=markers)
    m.add_layer(layer_group)
    return m

def plot_ts_data(df, x_attr, y_attr, plot_name, color=None):
    fig = px.line(
        df, 
        x=x_attr, 
        y=y_attr, 
        title=plot_name, 
        markers=True,
        color=color
    )
    return fig
    
def get_my_bookmarks():
    bookmarks=[{"name": "Tsunami History WFS (ITED V1)",
              "url" : "https://tsunamiarchiveservices.ingv.it/services/tsunami/wfs/wfs?service=WFS&version=2.0.0&request=GetFeature&typeNames=tsunami%3AITED_v1_localities&outputFormat=json&cql_filter=bbox%28geom%2C28.2829990386963,-56.3000030517578,69.8170013427734,41%29" },
              {"name": "Mt Etna Earthquake Parameters (2000-2019)",
              "url" : "https://vo-tcs.ct.ingv.it/rest/seismic-catalog/query/starttime=2018-10-22T21%3A37%3A52&endtime=2018-11-01T01%3A40%3A02?minlat=37.7&maxlat=37.78&minlong=14.993&maxlong=15.1&mdmin=1.1&mdmax=1.5&mlmin=1.1&mlmax=1.7&mindepth=4" },
               {"name": "TABOO Meteo at CO2 site - air temperature - uppiano",
                "url": "https://fridgews.ingv.it/meteo/?param_name=air_temperature&mintime=2016-12-01T00%3A00%3A00.000Z&maxtime=2016-12-31T23%3A59%3A00.000Z&minlat=43&maxlat=44&minlon=11.9662&maxlon=13&stacode=uppiano"},
               {"name": "TABOO Meteo at CO2 site - air temperature - nogna",
                "url": "https://fridgews.ingv.it/meteo/?param_name=air_temperature&mintime=2016-12-01T00%3A00%3A00.000Z&maxtime=2016-12-31T23%3A59%3A00.000Z&minlat=43&maxlat=44&minlon=11.9662&maxlon=13&stacode=nogna"}               
              ]
    return bookmarks


class Workspace:
    def __init__(self, bookmarks):
        self.bookmarks = bookmarks
        self.dataframe=pd.DataFrame(self.bookmarks)
        self.dataframe.style.set_properties(**{'width': '300px'}, **{'text-align': 'left'})
      
    def get_bookmarks(self):
        return self.bookmarks

    def get_bookmarks_df(self):
        bkdf=pd.DataFrame(self.bookmarks)
        bkdf.style.set_properties(**{'width': '300px'}, **{'text-align': 'left'})
        return bkdf

    def get_bookmark(self, i):
        return self.bookmarks[i]['name'], self.bookmarks[i]['url']

    def get_data(self, i):
        return get_from_url(self.bookmarks[i]['url'])

    def _repr_html_(self):
        #with pd.option_context('display.max_colwidth', None, 'display.colheader_justify', 'left'):
        return self.dataframe._repr_html_()

class IPSES:
    def __init__(self, userId):
        self.userId=userId

    def get_workspace(self):
        return Workspace( bookmarks=[{"name": "Tsunami History WFS (ITED V1)",
              "url" : "https://tsunamiarchiveservices.ingv.it/services/tsunami/wfs/wfs?service=WFS&version=2.0.0&request=GetFeature&typeNames=tsunami%3AITED_v1_localities&outputFormat=json&cql_filter=bbox%28geom%2C28.2829990386963,-56.3000030517578,69.8170013427734,41%29" },
              {"name": "Mt Etna Earthquake Parameters (2000-2019)",
              "url" : "https://vo-tcs.ct.ingv.it/rest/seismic-catalog/query/starttime=2018-10-22T21%3A37%3A52&endtime=2018-11-01T01%3A40%3A02?minlat=37.7&maxlat=37.78&minlong=14.993&maxlong=15.1&mdmin=1.1&mdmax=1.5&mlmin=1.1&mlmax=1.7&mindepth=4" },
               {"name": "TABOO Meteo at CO2 site - air temperature - uppiano",
                "url": "https://fridgews.ingv.it/meteo/?param_name=air_temperature&mintime=2016-12-01T00%3A00%3A00.000Z&maxtime=2016-12-31T23%3A59%3A00.000Z&minlat=43&maxlat=44&minlon=11.9662&maxlon=13&stacode=uppiano"},
               {"name": "TABOO Meteo at CO2 site - air temperature - nogna",
                "url": "https://fridgews.ingv.it/meteo/?param_name=air_temperature&mintime=2016-12-01T00%3A00%3A00.000Z&maxtime=2016-12-31T23%3A59%3A00.000Z&minlat=43&maxlat=44&minlon=11.9662&maxlon=13&stacode=nogna"}               
              ])
        

In [ ]:
import pandas as pd
import random
import requests
import pandas as pd
import plotly.graph_objects as go
from datetime import datetime, timedelta
from IPython.display import display, clear_output
import threading
import time

class TemperatureDataSimulator:
    def __init__(self, start_time=None, interval_seconds=10, window_size=10):
        self.window_size = window_size
        self.interval_seconds = interval_seconds
        self.data = pd.DataFrame(columns=['time', 'value'])
        initial_time = start_time if start_time else datetime.now()
        initial_temperature = random.uniform(-20, 20)
        initial_data = pd.DataFrame([{'time': initial_time, 'value': initial_temperature}])
        self.data = pd.concat([self.data, initial_data], ignore_index=True)
        self.lock = threading.Lock()

    def fetch_new_data_point(self):
        with self.lock:
            last_time = self.data['time'].iloc[-1]
            new_time = last_time + timedelta(seconds=self.interval_seconds)
            new_temperature = random.uniform(-20, 20)
            new_data_point = pd.DataFrame([{'time': new_time, 'value': new_temperature}])
            self.data = pd.concat([self.data, new_data_point], ignore_index=True)
            if len(self.data) > self.window_size:
                self.data = self.data.iloc[-self.window_size:]			

class DisplayDynamicDataDemo:
    def __init__(self, simulator):
        self.simulator=simulator
        self.stop_event = threading.Event()

    def fetch_data(self, mintime, maxtime):
        self.simulator.fetch_new_data_point()
        ts_dataframe=self.simulator.data
        return ts_dataframe

    def initialize_plot(self):
        self.fig = go.Figure()
        self.fig.update_layout(title='Dynamic data - temperature (simulated)', xaxis_title='time', yaxis_title='value')
        return self.fig

    def update_plot(self, interval_minutes):
        while not self.stop_event.is_set():
            now = datetime.utcnow()
            mintime = (now - timedelta(minutes=interval_minutes)).strftime('%Y-%m-%dT%H:%M:%S')
            maxtime = now.strftime('%Y-%m-%dT%H:%M:%S')
            
            df = self.fetch_data(mintime, maxtime)
            
            if not df.empty:
                self.fig.data = []
                self.fig.add_trace(go.Scatter(x=df['time'], y=df['value'], mode='lines', name='Time Series'))
                clear_output(wait=True)
                display(self.fig)
            else:
                print("No new data to display.")
            
            time.sleep(interval_minutes * 60)  # Wait for the next update

    def start_display(self, sample_time_minutes=0.05):
        self.fig = self.initialize_plot()
        display(self.fig)
        
        update_thread = threading.Thread(target=self.update_plot,  kwargs={'interval_minutes': sample_time_minutes})
        update_thread.start()

    def stop_display(self):
        self.stop_event.set()

In [ ]:
# retrieve the personal workspace (fake)

In [ ]:
my_workspace=IPSES('UserId1').get_workspace()

In [ ]:
################################################################################################################################
# My current workspace:
################################################################################################################################

In [ ]:
my_workspace

In [ ]:
################################################################################################################################
# Example 1 - Retrieve data from Italian Tsunami Effects Database: Tsunami History WFS (ITED V1)
################################################################################################################################

In [ ]:
# retrieve the remote json data from the first 'bookmark' defined in my workspace
tsunami_data=json.loads(my_workspace.get_data(0))
# Make it a flat table and create a dataframe
tsunami_dataframe=pd.json_normalize(tsunami_data['features'])[['properties.ISTAT', 'properties.Link', 'properties.Municipality', 
                                                               'properties.PlaceID', 'properties.PlaceName', 'properties.Province', 
                                                               'properties.PlaceLat', 'properties.PlaceLon']]

In [ ]:
# Save the dataframe to a CSV file, for further analyses
tsunami_dataframe.to_csv('tsunami_data_01.csv')

In [ ]:
tsunami_dataframe

In [ ]:
# Display the dataframe on a map (through the ipyleaflet widget)
show_tsunami_data_on_a_map(tsunami_dataframe)

In [ ]:
# Let's filter the dataframe to keep just some provinces, using pandas
tsunami_dataframe_filtered = tsunami_dataframe[tsunami_dataframe['properties.Province'].isin(['FC', 'RA', 'FE', 'RN'])]

In [ ]:
# Display it on a map
show_tsunami_data_on_a_map(tsunami_dataframe_filtered)

In [ ]:
################################################################################################################################
# Example 2: Retrieve data from Mt Etna Earthquake Parameters (2000-2019) 
################################################################################################################################

In [ ]:
volcano_data=json.loads(my_workspace.get_data(1))
volcano_dataframe=pd.DataFrame(volcano_data)

In [ ]:
# Save the dataframe to a CSV file
volcano_dataframe.to_csv('volcano_data_01.csv')

In [ ]:
volcano_dataframe

In [ ]:
# Display the dataframe on a map (through the ipyleaflet widget)
show_volcano_data_on_a_map(volcano_dataframe)

In [ ]:
# try update the dataframe with a new (fake) entry and display it again on a map

last_row = volcano_dataframe.iloc[-1]
last_row['site']='Milan - monte stella'
last_row['lat']=45.490132926198754
last_row['lon']=9.133201710252422
silly_dataframe = pd.concat([volcano_dataframe, pd.DataFrame([last_row])], ignore_index=True)
show_volcano_data_on_a_map(silly_dataframe, zoom_level=5)

In [ ]:
################################################################################################################################
# Example 3 - Retrieve time series data (air temperature),  from TABOO Meteo at CO2 site
################################################################################################################################

In [ ]:
# retrieve the data from the remote service and build a Dataframe out of it
ts_data=json.loads(my_workspace.get_data(2))
ts_dataframe=pd.json_normalize(pd.DataFrame(ts_data).results)

In [ ]:
# Save the dataframe to a CSV file
ts_dataframe.to_csv('ts_data_01.csv')

In [ ]:
ts_dataframe

In [ ]:
# Plot the graph for this TS (through the plotly widgets library)
plot_ts_data(ts_dataframe, 'time', 'value', 'air temperature', color='station_code')

In [ ]:
# Let's ovedrlay an additional time series
# retrieve the new TS amd create a dataframe
ts_data2=json.loads(my_workspace.get_data(3))
ts_dataframe2=pd.json_normalize(pd.DataFrame(ts_data2).results)

In [ ]:
# then, plot the TSs
plot_ts_data(pd.concat([ts_dataframe, ts_dataframe2]), 'time', 'value', 'air temperature from multiple stations', color='station_code')

In [ ]:
################################################################################################################################
# Example 4 - demo dynamic data (fake): updates each 3 seconds
################################################################################################################################

In [ ]:
dyn_graph1=DisplayDynamicDataDemo(TemperatureDataSimulator(window_size=10, interval_seconds=10))
dyn_graph1.start_display(sample_time_minutes=0.05)

In [ ]:
# to stop updating the graph
dyn_graph1.stop_display()